In [16]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler


In [17]:
#loading the dataset
data = pd.read_csv('Tic tac.csv')

In [18]:
#displaying the first 5 rows of the dataset
data.head()

,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,?,?,?,loss
1,4,7,2,6,?,?,?,win
2,0,8,1,6,5,?,?,draw
3,4,7,2,3,?,?,?,draw
4,0,4,2,1,?,?,?,win


In [19]:
#checking for missing values
data.isnull().sum()


MOVE1    0
MOVE2    0
MOVE3    0
MOVE4    0
MOVE5    0
MOVE6    0
MOVE7    0
CLASS    0
dtype: int64

In [20]:
#checking for unique values in the dataset
data.nunique()

MOVE1     9
MOVE2    10
MOVE3    10
MOVE4    10
MOVE5    10
MOVE6     8
MOVE7     5
CLASS     3
dtype: int64

In [21]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [22]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# Tic tac.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('Tic tac.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'Tic tac.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')



There are 1000 rows and 8 columns


In [23]:
#reinforment learning model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [24]:
#splitting the dataset into independent and dependent variables
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [25]:
#splitting the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [28]:
# Assuming your data is in a DataFrame named 'df'
# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

# Impute missing values with mean or median
df.fillna(df.mean(), inplace=True)

NameError: name 'df' is not defined